## Más sobre funciones


El análisis de datos en el mundo real suele implicar una cuidadosa y detallada transformación y agregación de datos, que puede realizarse con una gran variedad de funciones, ya sean parte del paquete base  o proporcionadas por paquetes de extensión. Para utilizar mejor estas funciones se  necesita una comprensión básica pero concreta de cómo funcionan las funciones `R`. 

### Evaluación perezosa

Una gran parte de la comprensión de cómo R funciona se puede hacer, averiguando cómo trabajan las funciones. Supongamos que creamos la siguiente función:

In [1]:
prueba0 <- function(x, y) {
    if (x > 0) x else y
}

La función es algo especial porque `y` parece ser necesario sólo cuando `x` es menor que cero. ¿Qué pasa si sólo suministramos un número positivo a `x` e ignoramos `y`? ¿Fallará la función porque no suministramos todos los argumentos en su definición? Averigüemos llamando a la  función:

In [2]:
prueba0(1)

[1] 1

La función funciona sin que  se suministre la variable `y`. Parece que no estamos obligados a suministrar los valores a todos los argumentos cuando llamamos a una función, sino sólo a aquellos que son necesarios. Si llamamos a `prueba0` con un número negativo, `y` es necesario:

In [3]:
prueba0(-2)

ERROR: Error in prueba0(-2): el argumento "y" está ausente, sin valor por omisión


Como no especificamos el valor de `y`, la función se detiene, informando la falta de `y`.

De los ejemplos anteriores, se aprende que una función no requiere que se especifiquen todos los argumentos si no son necesarios para devolver un valor. ¿Qué pasa si insistimos en especificar los argumentos que no se utilizan en la función? ¿Serán evaluados antes de llamar a la función o no son  evaluados en absoluto?

Averiguémos poniendo una función `stop()` en la posición del argumento `y`. Si la expresión se evalúa por cualquier medio, debe detenerse inmediatamente antes de que se devuelva `x`:

In [4]:
prueba0(1, stop("Nos detenemos ahora"))

[1] 1

La salida indica que `stop()` no ocurre, lo que indica que no se evalúa en absoluto. Si cambiamos el valor de `x` a un número negativo, la función debería detenerse:

In [5]:
prueba0(-1, stop("Nos detenemos ahora"))

ERROR: Error in prueba0(-1, stop("Nos detenemos ahora")): Nos detenemos ahora


Ahora, es claro que `stop()` se evalúa en este caso. El mecanismo se vuelve bastante transparente. En una llamada de función, la expresión de un argumento se evalúa sólo cuando el valor del argumento es necesario. Este mecanismo se denomina **evaluación perezosa**  y por lo tanto, también podemos decir, que los argumentos de una llamada de función son evaluados perezosamente, es decir, evaluados sólo cuando es necesario.

Si no estas al tanto del mecanismo de evaluación perezosa, se puede pensar que la llamada de función siguiente debe consumir mucho tiempo y puede agotar toda la memoria del equipo. Sin embargo, la evaluación perezosa evita que suceda porque `rnorm(1000000)` nunca se evalúa. Esto se debe a que nunca se necesita cuando se evalúa `if (x> 0) x else y`, lo que se puede verificar sincronizando las llamadas de función, usando `system.time()`:

In [6]:
system.time(rnorm(10000000))

   user  system elapsed 
  3.020   0.016   3.072 

Generar `10` millones de números aleatorios no es un trabajo fácil. Se tarda más de un segundo. Por el contrario, la evaluación de un número debe ser más fácil en `R` y es tan rápido que el temporizador en sí no puede decirnos mucha información:

In [7]:
system.time(1)

   user  system elapsed 
      0       0       0 

Si consideramos la siguiente expresión, dada la lógica de `prueba0` y el conocimiento de la evaluación perezosa, `system.time` debe indicar cero:


In [8]:
system.time(prueba0(1, rnorm(10000000)))

   user  system elapsed 
      0       0       0 

Otro escenario de evaluación perezosa que podría ocurrir son los valores predeterminados de los argumentos. Más precisamente, los valores por defecto de los argumentos de la función deben ser realmente expresiones por defecto porque el valor no está disponible hasta que la expresión realmente se evalúa.

Consideramos la siguiente función:

In [9]:
prueba1 <- function(x, y = stop("paramos ahora")) {
    if (x > 0) x else y
}

Le damos a  `y` un valor por defecto que llama a `stop()`. Si la evaluación perezosa no se aplica aquí, es decir, si `y` es evaluado independientemente de si es necesario, deberíamos recibir un error mientras llamamos a `prueba1()` sin suministrar `y`. Sin embargo, si la evaluación perezosa se aplica, llamar a `prueba1()` con un argumento `x` positivo no debería causar un error ya que la expresión `stop()` de `y` nunca se evalúa.

Hagamos un experimento para averiguar cuál es la verdad. En primer lugar, llamaremos `prueba1()` con un argumento `x` positivo:

In [10]:
prueba1(1)

[1] 1

La salida implica que la evaluación perezosa también funciona aquí. La función sólo utiliza `x`, y la expresión por defecto de `y` no se evalúa en absoluto. Si suministramos un argumento `x` negativo en su lugar, la función debería detenerse como se supone:

In [11]:
prueba1(-3)

ERROR: Error in prueba1(-3): paramos ahora


Los ejemplos anteriores demuestran una ventaja de la evaluación perezosa: hace posible ahorrar tiempo y evitar la evaluación innecesaria de las expresiones. Además, también permite una especificación más flexible de los valores por defecto de los argumentos de la función. Por ejemplo, se puede utilizar otros argumentos en la expresión de un argumento de función:

In [12]:
prueba2 <- function(x, n = floor(length(x) / 2)) {
    x[1:n]
}

Esto  permite configurar el comportamiento predeterminado de una función de una manera más razonable o deseable, mientras que los argumentos de la función siguen siendo tan personalizables como estaban sin esos valores predeterminados.

Si llamamos a `prueba2`, sin especificar `n`, el comportamiento por defecto extrae los elementos de la primera mitad de `x`:

In [13]:
prueba2(1:10)

[1] 1 2 3 4 5

La función sigue siendo flexible ya que siempre se puede anular su comportamiento predeterminado especificando otro valor de `n`:

In [14]:
prueba2(1:10, 3)

[1] 1 2 3

Como todas las demás características, la evaluación perezosa también tiene sus pros y sus contras. Dado que los argumentos de una función sólo se analizan pero no se evalúan cuando se llama a la función, sólo podemos asegurarnos de que las expresiones suministradas a los argumentos son sintácticamente correctas. Es difícil asegurar que los argumentos van a funcionar.

Por ejemplo, si una variable indefinida aparece en el valor predeterminado de un argumento, no habrá ninguna advertencia o error en el momento en que creamos la función. En el ejemplo siguiente, creamos una función `prueba3`, que es exactamente igual que `prueba2`, excepto que `x` en `n` se escribe erróneamente como una variable indefinida `m`.

In [15]:
prueba3 <- function(x, n = floor(length(m) / 2)) {
    x[1:n]
}

Cuando creamos `prueba3`, no hay ninguna advertencia o error porque `floor(length (m)/2)` nunca se evalúa antes de que se llame a `prueba3` y el valor de `n` es demandado por `1: n`. La función se detendrá sólo cuando la llamemos:

In [16]:
prueba3(1:10)

ERROR: Error in prueba3(1:10): objeto 'm' no encontrado


Si hemos definido `m` antes de que se llame a `prueba3`, la función trabaja, pero de una manera inesperada:

In [17]:
m <- c(1, 2, 3)
prueba3(1:10)

[1] 1

Otro ejemplo que hace que la evaluación de la pereza sea más explícita es el siguiente:

In [18]:
prueba4 <- function(x, y = p) {
    p <- x + 1
    c(x, y)
}

Se debe tener  en cuenta que el valor predeterminado de `y` es `p`, que no se define antes de que se llame a la función, como en el ejemplo anterior. Una diferencia notable entre estos dos ejemplos es cuando se suministra el símbolo que falta en el valor predeterminado del segundo argumento. En el ejemplo anterior, `p` se define antes de llamar a la función. Sin embargo, en este ejemplo, `p` se define dentro de la función antes de que se utilice `y`.

Veamos qué sucede cuando llamamos a la función:

In [19]:
prueba4(1)

[1] 1 2

Parece que la función funciona en lugar de terminar en un error. Será más fácil de entender si pasamos por el proceso detallado de cómo `prueba4(1)` es ejecutada:

* Se encuentra una función llamada `prueba4`.
* Coincidimos los argumentos dados, pero tanto `x` como `y` no se evalúan.
* `p <- x + 1` evalúa `x + 1` y asigna el valor a una nueva variable `p`.
* `c(x, y)` evalúa tanto `x` como `y`, donde `x` toma `1` e  `y` toma `p`, lo cual sucede para obtener el valor de `x + 1`, que es `2`.
* La función retorna un vector numérico `c(1,2)`.

Por lo tanto, en todo el proceso de evaluación de `prueba4(1)`, no se produce ninguna advertencia o error porque no se infringen reglas. El truco más importante aquí es que `p` sólo se define antes de que se utilice `y`.

El ejemplo anterior ayuda a explicar cómo funciona la evaluación perezosa, pero de hecho es una mala práctica. No recomiendo escribir una función de esta manera porque este truco sólo hace que el comportamiento de la función sea menos transparente. Una buena práctica es simplificar los argumentos y evitar el uso de símbolos indefinidos fuera de la función. De lo contrario, puede ser difícil predecir su comportamiento o depurar la función debido a su dependencia del entorno externo.

A pesar de esto, hay un cierto uso sabio de la evaluación perezosa también. Por ejemplo, `stop()`,  se puede utilizar junto con `switch()` en el último argumento para hacer que la función se detenga cuando no coinciden los casos. La siguiente función `prueba_entrada()` utiliza `switch()` para regular la entrada de `x` de modo que sólo acepta `y` e `n`. La función  se detiene cuando se suministran otras cadenas:

In [20]:
prueba_entrada <- function(x) {
    switch(x,
           y = message("si"),
           n = message("no"),
           stop("Entrada invalida"))
}

Cuando `x` toma `y`, un mensaje diciendo `si` se  muestra:

In [21]:
prueba_entrada("y")

si


Cuando `x` toma `n`, aparece un mensaje diciendo `no`:

In [22]:
prueba_entrada("n")

no


De lo contrario, la función se detiene:

In [23]:
prueba_entrada("Que es esto?")

ERROR: Error in prueba_entrada("Que es esto?"): Entrada invalida


El ejemplo funciona porque `stop()` es evaluado perezosamente como un argumento de `switch()`.

No se puede confiar demasiado en el analizador para comprobar el código. Sólo comprueba el código en su sintaxis y no indica si el código está escrito con buenas prácticas. Para evitar los peligros potenciales causados por la evaluación perezosa, haga la comprobación necesaria en la función para cerciorarse de que la entrada se puede manejar correctamente.

### El mecanismo de copiar-poner-modificar

En la parte anterior  mostramos cómo funciona la evaluación perezosa y cómo puede ayudar a ahorrar tiempo de computación y  trabajo de memoria al evitar la evaluación innecesaria de los argumentos de la función. En esta parte se mostrará una característica importante de R que hace que sea más seguro trabajar con datos. 

Supongamos que creamos un vector numérico simple `x1`:

In [24]:
x1 <- c(1, 2, 3)

A continuación, asignamos el valor de `x1` a `x2`:

In [25]:
x2 <- x1

Ahora, `x1` y `x2` tienen exactamente el mismo valor. ¿Qué pasa si modificamos un elemento en uno de los dos vectores? ¿Cambiarán ambos vectores?

In [26]:
x1[1] <- 0
x1

[1] 0 2 3

In [27]:
x2

[1] 1 2 3

La salida muestra que cuando `x1` cambia, `x2` permanecerá sin cambios. Se puede adivinar que la asignación copia automáticamente el valor y hace que la nueva variable apunte a la copia de los datos en lugar de los datos originales. Vamos a usar `tracemem()` para rastrear la huella de los datos en la memoria.

Vamos a restablecer los vectores y realizar un experimento mediante el seguimiento de las direcciones de memoria de `x1` e  `x2`:

In [28]:
x1 <- c(1, 2, 3)
x2 <- x1

Llamamos `tracemem()` en los dos vectores, que muestra la dirección de memoria actual de los datos. Si cambia la dirección de memoria que se está rastreando,  aparecerá un texto con la dirección original y la nueva dirección, indicando que la data es copiada:

In [29]:
tracemem(x1)

[1] "<0x40be450>"

In [30]:
tracemem(x2)

[1] "<0x40be450>"

Ahora, ambos vectores tienen el mismo valor y `x1` e  `x2` comparten la misma dirección, lo que implica que apuntan exactamente a la misma parte  de datos en la memoria y que la operación de asignación no copia los datos automáticamente. Pero, ¿cuándo se copian los datos?

Ahora, modificaremos el primer elemento de `x1` a `0`:

In [31]:
x1[1] <- 0

tracemem[0x40be450 -> 0x3bd82c8]: eval eval withVisible withCallingHandlers doTryCatch tryCatchOne tryCatchList tryCatch try handle timing_fn evaluate_call evaluate doTryCatch tryCatchOne tryCatchList doTryCatch tryCatchOne tryCatchList tryCatch <Anonymous> handle_shell <Anonymous> <Anonymous> 


El rastreo de memoria indica que la dirección de `x1` ha cambiado a una nueva.  Ahora tenemos dos copias de los mismos datos en dos lugares diferentes. A continuación, se modifica el primer elemento de una copia y finalmente, `x1`  se acomoda para  apuntar  a la copia modificada.

Ahora, `x1` y `x2` tienen valores diferentes: `x1` apunta al vector modificado y `x2` permanece apuntando al vector original. En otras palabras, si varias variables se refieren al mismo objeto, modificar una variable hará una copia del objeto. Este mecanismos es llamado **copiar-poner-modificar**.

Otro escenario donde el mecanismo **copiar-poner-modificar** sucede es cuando modificamos un argumento de una  función. Supongamos que creamos la siguiente función:

In [32]:
modifica_primero <- function(x) {
    x[1] <- 0
    x
}

Cuando se ejecuta la función, se intenta modificar el primer elemento del argumento `x`. Hagamos algunos experimentos con vectores y listas y veamos si `modifica_primero()` puede modificarlos.

Para un vector numérico `v1`:

In [33]:
v1 <- c(1, 2, 3)
modifica_primero(v1)

[1] 0 2 3

In [34]:
v1

[1] 1 2 3

Para una lista `v2`:

In [35]:
v2 <- list(x = 1, y = 2)
modifica_primero(v2)

$x
[1] 0

$y
[1] 2

In [36]:
v2

$x
[1] 1

$y
[1] 2

En ambos experimentos, la función sólo devolvió una versión modificada del objeto original, pero no modificó el objeto original. Sin embargo, la modificación directa de los vectores fuera de la función trabaja:

In [37]:
v1[1] <- 0
v1

[1] 0 2 3

In [38]:
v2[1] <- 0
v2

$x
[1] 0

$y
[1] 2

Para usar la versión modificada, necesitamos asignarla a la variable original:

In [39]:
v3 <- 1:5
v3 <- modifica_primero(v3)
v3

[1] 0 2 3 4 5

Los ejemplos anteriores demuestran que la modificación de un argumento de función también hace que una copia asegure de que la modificación no afecta a las cosas fuera de la función.

El mecanismo copiar-poner-modificar también ocurre cuando se modifican los atributos. La siguiente función elimina los nombres de fila de un data frame y reemplaza sus nombres de columna con mayúsculas:

In [40]:
cambiar_nombre <- function(x) {
    if (is.data.frame(x)) {
        rownames(x) <- NULL
        if (ncol(x) <= length(LETTERS)) {
            colnames(x) <- LETTERS[1:ncol(x)]
        } else {
            stop("Muchas columnas para renombrar")
        }
    } else {
        stop("x debe ser un data frame")
    }
    x
}

Para probar la función, crearemos un data frame simple con datos generados aleatoriamente:

In [41]:
df_simple <- data.frame(
    id = 1:3,
    ancho = runif(3, 5, 10),
    altura = runif(3, 5, 10))
df_simple

id,ancho,altura
1,7.838903,9.938184
2,9.410789,9.274761
3,6.395929,5.671547


Ahora, llamaremos la función con el data frame y veremos la versión modificada:

In [42]:
cambiar_nombre(df_simple)

A,B,C
1,7.838903,9.938184
2,9.410789,9.274761
3,6.395929,5.671547


Según el mecanismo `copiar-poner-modificar`, `df_simple` se copia la primera vez cuando se remueven los  nombres de las fila y a continuación, se realizan todos los cambios subsiguientes en la versión copiada en lugar de la versión original. Podemos verificar esto viendo `df_simple`:

In [43]:
df_simple

id,ancho,altura
1,7.838903,9.938184
2,9.410789,9.274761
3,6.395929,5.671547


La versión original no ha cambiado en absoluto.

### Modificación de objetos fuera de una función

A pesar del mecanismo copiar-poner-modificar, todavía es posible modificar un vector fuera de una función. El operador `<<-` está diseñado para hacer el trabajo. Supongamos que tenemos una variable `x` y creamos una función `modifica_x()` que simplemente asigna un nuevo valor a `x`:

In [44]:
x <- 0
modifica_x <- function(valor) {
    x <<- valor
}

Cuando llamamos a la función, el valor de `x` será reemplazado:

In [45]:
modifica_x(3)
x

[1] 3

Esto puede ser útil  cuando se intenta asignar un vector a una nueva lista y hacer un conteo al mismo tiempo. El siguiente código crea una lista de vectores con un número creciente de elementos. En cada iteración de `lapply()`, la variable `conteo` se utiliza para sumar el número total de elementos en el vector generado:

In [46]:
conteo <- 0
lapply(1:3, function(x) {
    resultado <- 1:x
    conteo <<- conteo + length(resultado)
    resultado
})

[[1]]
[1] 1

[[2]]
[1] 1 2

[[3]]
[1] 1 2 3

In [47]:
conteo

[1] 6

Otro ejemplo en el que `<<-` es útil para 'aplanar' una lista anidada. Supongamos que tenemos una lista anidada como la que se muestra aquí:

In [48]:
lista_anidada <- list(
    a = c(1, 2, 3),
    b = list(
        x = c("a", "b", "c"),
        y = list(
            z = c(TRUE, FALSE),
            w = c(2, 3, 4))
    )
)
str(lista_anidada)

List of 2
 $ a: num [1:3] 1 2 3
 $ b:List of 2
  ..$ x: chr [1:3] "a" "b" "c"
  ..$ y:List of 2
  .. ..$ z: logi [1:2] TRUE FALSE
  .. ..$ w: num [1:3] 2 3 4


Queremos aplanar la lista para que los niveles anidados se lleven al primer nivel. El siguiente código resuelve el problema usando `rapply()` y  `<<-`.

In [49]:
help(rapply)


Primero, necesitamos saber que `rapply()` es una versión recursiva de `lapply()`. En cada iteración, la función suministrada se llama con un vector atómico en un nivel particular de la lista hasta que todos los vectores atómicos en todos los niveles se agotan. Llamar a `rapply(lista_anidada, f)` básicamente funciona de la siguiente manera:

```

f(c(1, 2, 3))
f(c("a", "b", "c"))
f(c(TRUE, FALSE))
f(c(2, 3, 4))
```

Debemos tener en cuenta, que debemos elaborar una solución para aplastar `lista_anidada`. La solución que vamos a analizar está inspirada en una respuesta de [stackoverflow](https://stackoverflow.com/questions/8139677/how-to-flatten-a-list-to-a-list-without-coercion/8139959#8139959), que utiliza inteligentemente `rapply()`.

Primero, crearemos una lista vacía para recibir vectores individuales en la lista anidada y un contador:

In [50]:
lista_plana <- list()
i <- 1

Luego, usaremos `rapply()` para aplicar recursivamente una función a `lista_anidada`. En cada iteración, la función recibe un vector atómico en `lista_anidada` a través de `x`. La función establece el  `iesimo` elemento de  la función `lista_plana` en `x` e incrementa el contador `i`:

In [51]:
resultado1 <- rapply(lista_anidada, function(x) {
    lista_plana[[i]] <<- x
    i <<- i + 1
})

Con las iteraciones realizadas, todos los vectores atómicos se almacenan en `lista_plana` en el primer nivel. El valor devuelto por `rapply()` es el siguiente:

In [52]:
resultado1

a   b.x b.y.z b.y.w 
    2     3     4     5

Como resultado de `i <<- i + 1`, los valores en `resultado1` no tienen mucha importancia. Sin embargo, los nombres de `resultado1` son útiles para indicar los niveles y nombres originales de cada elemento en `lista_plana`. Así que dejamos que `lista_plana` también tenga los nombres de `resultado1` para indicar el origen de cada elemento:

In [53]:
names(lista_plana) <- names(resultado1)
str(lista_plana)

List of 4
 $ a    : num [1:3] 1 2 3
 $ b.x  : chr [1:3] "a" "b" "c"
 $ b.y.z: logi [1:2] TRUE FALSE
 $ b.y.w: num [1:3] 2 3 4


Finalmente, todos los elementos de `lista_anidada`, se almacenan de forma plana en `lista_plana`.